In [ ]:
from collections import defaultdict

import requests
from tqdm import tqdm

In [ ]:
resp = requests.get("https://gist.githubusercontent.com/dracos/dd0668f281e685bad51479e5acaadb93/raw/6bfa15d263d6d5b63840a8e5b64e04b382fdb079/valid-wordle-words.txt")

In [ ]:
with open("words.txt") as f:
    words = {word for line in f.readlines() for word in line.strip().lower().split() if len(word)==5}

In [ ]:
def ori_tuple(word, guess):
    rtuple = list(0 for _ in range(len(guess)))
    guess_rest = {}
    word_rest = {}
    for i,l in enumerate(guess):
        if word[i]==l:
            rtuple[i] = 2
        else:
            guess_rest[i]=l
            word_rest[i]=word[i]
    for wl in set(word_rest.values()) & set(guess_rest.values()):
        c = list(word_rest.values()).count(l)
        for i,gl in guess_rest.items():
            if wl==gl:
                rtuple[i] = 1
                c-=1
                if c==0:
                    break
    return tuple(rtuple)

In [ ]:
def gen_wordle(words:set[str], guesses):
    notin:set[str] = set()
    for guess, guess_tuple in guesses.items():
        for l in set(guess):
            cl = sum(guess_tuple[ix]>=1 for ix in range(len(guess)) if guess[ix]==l)
            if all(guess_tuple[ix]>=1 for ix in range(len(guess)) if guess[ix]==l):
                for word in words-notin:
                    if cl>word.count(l):
                        notin.add(word)
            else:
                for word in words-notin:
                    if cl!=word.count(l):
                        notin.add(word)
        for i,xl in enumerate(guess_tuple):
            for word in words-notin:
                if xl==2 and guess[i]!=word[i]:
                    notin.add(word)
                elif xl!=2 and guess[i]==word[i]:
                    notin.add(word)
    return words - notin

In [ ]:
def next_wordle(words, no_bar=False):
    rdict = defaultdict(lambda: defaultdict(int))
    for word in tqdm(words, disable=no_bar):
        for guess in words:
            rdict[guess][ori_tuple(word, guess)] += 1
    score = {guess:max(rdict[guess][tup] for tup in rdict[guess]) for guess in rdict}
    return set(filter(lambda word:score[word]==min(score.values()), score))

In [ ]:
next_wordle(
    gen_wordle(
        words,
        {
            "arise": (0,0,0,0,0),
        }
    ),
    no_bar=False
)